In [ ]:
function solve_model(K,S,C,D,X,y0,d, lambda)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 100)

    n_stations, _, n_hours = size(d)

    # Decision variables
    @variable(model, x[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Bin)
    @variable(model, 0 <= z[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Int)
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints:
    # Stations capacity
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    # No rebalancing at the same station
    @constraint(model, [i in 1:n_stations, k in 1:K, t in 1:n_hours], x[i,i,k,t] == 0)
    # Flow balance 
    @constraint(model, [i in 1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)-sum(z[i,j,k,t] for j in 1:n_stations, k in 1:K)+sum(z[j,i,k,t] for j in 1:n_stations, k in 1:K))
    # Flow balance for the first hour 
    @constraint(model, [i in 1:n_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations)-sum(z[i,j,k,1] for j in 1:n_stations, k in 1:K)+sum(z[j,i,k,1] for j in 1:n_stations, k in 1:K))
    # Vans capacity
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S)
    # Users travel availability
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    # Max 1 rebalancing per hour per van
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations) <= 1)
    # Rebalancing feasibility
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S*x[i,j,k,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours], x[i,j,k,t] <= X[i,j])
    # Users travel is inferior to the d and definition of u
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], d[i,j,t] >= w[i,j,t])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)+lambda*sum(D[i,j]*x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

In [ ]:
function export_results(z)
    z_nonzero = Tuple.(findall(x -> x != 0, z))[:,:]
    nb_rebal = size(z_nonzero)[1]
    new=zeros(nb_rebal,5)
    for i=1:nb_rebal
        indices=collect(z_nonzero[i])
        new[i,1:4]=indices # indices of rebalancing
        new[i,5]=z[indices[1],indices[2],indices[3],indices[4]] # nb of bikes
    end
    return new
end

In [ ]:
function export_sequential_results(z,v)
    z_nonzero = Tuple.(findall(x -> x != 0, z))[:,:]
    nb_rebal = size(z_nonzero)[1]
    new=zeros(nb_rebal,5)
    for i=1:nb_rebal
        indices=collect(z_nonzero[i])
        new[i,1:4]=indices # indices of rebalancing
        new[i,5]=z[indices[1],indices[2],indices[3],indices[4]] # nb of bikes
    end
    v_nonzero = Tuple.(findall(x -> x != 0, v))[:,:]
    nb_rebal = size(v_nonzero)[1]
    new2=zeros(nb_rebal,5)
    for i=1:nb_rebal
        indices=collect(v_nonzero[i])
        new2[i,1:4]=indices # indices of rebalancing
        new2[i,5]=0
    end
    return vcat(new,new2)
end

In [ ]:
function print_results(results)
    for i=1:size(results)[1]
        println("Rebalancing from station ",results[i,1], " to ", results[i,2], "by van ", results[i,3], " at hour ", results[i,4]," of ", results[i,5]," bikes ")
    end
end

In [ ]:
# DUMMY DATA 
# capacity of stations
C = [10,10,10,10];

# d of stations
d = [[[0,10,0,5] [0,0,0,0] [0,5,0,0] [20,0,0,0]];;;
    [[0,0,0,0] [0,0,0,0] [30,0,0,0] [0,10,0,0]];;;
    [[0,0,0,5] [0,0,15,0] [0,0,0,0] [0,0,0,0]]];

# proximity of stations
X = [[0,1,1,1] [1,0,1,1] [1,1,0,1] [1,1,1,0]];

# capacity of stations
y0 = [6,2,4,3];

# number of vans
K = 2;

# capacity of vans
S = 20;

# distance of stations
D = [[0,0.5,1,0.5] [2,0,0.5,1] [0.5,1,0,0.5] [1,2,1,0]];

# number of stations
n_stations = 4;
n_hours=3;

In [ ]:
function create_d_bis(d_1,d_2)
    diff = d_2 - d_1
    n_stations = size(d_1,1)
    d_bis=zeros(n_stations,n_stations,24)
    for i in 1:n_stations
        for j in 1:n_stations
            for l in 1:24
                ecart=floor(diff[i,j,l]/4)
                if ecart>0
                    d_bis[i,j,l] = d_1[i,j,l] + rand(0:ecart)
                else
                    d_bis[i,j,l] = d_1[i,j,l] + rand(ecart:0)
                end
            end
        end
    end
    return d_bis
end